# Convert the Dataset into Vectors

The goal of this notebook is to generate a vector for each cell for all notebooks in the sliced-notebooks dataset.

Dimensions of vector array: n * sequence count * 300

# Import modules

In [4]:
import pandas as pd
import numpy as np
import os
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
import torch

# Import dataset and doc2vec model

In [5]:
df = pd.read_json("../data/all-notebooks.json", orient='index')
model = Doc2Vec.load("../model/notebook-doc2vec-model-mar24.model")

In [6]:
df.head(4)

,cell_type,source,filename,competition
0,code,from subprocess import check_output\n import n...,1806927,favorita-grocery-sales-forecasting
1,code,holiday = pd.read_csv('../input/holidays_event...,1806927,favorita-grocery-sales-forecasting
2,code,# converting date into datetime format\n holid...,1806927,favorita-grocery-sales-forecasting
3,code,# all bridge-type has puente (puente actually ...,1806927,favorita-grocery-sales-forecasting


# Group the dataset by notebook and generate doc2vec vectors

In [9]:
df_test = df

In [11]:
allVectors = []
allVectorsFilenames = []
for i, notebook in df_test.groupby("filename"):
    
    vectorSeq = []
    # vectorSeq is a list of doc2vec vectors corresponding to [Cell0, Cell1, .... Celln]
    # each vectorSeq list corresponds to a single notebook
    for j, row in notebook.iterrows():
        source = row[1]
        vector = model.infer_vector(source.split(" "))
        vectorSeq.append(vector)
    allVectors.append(vectorSeq)
    allVectorsFilenames.append(notebook.iloc[0]['filename'])

In [51]:
print(len(allVectors))

26430


# Save the arrays

In [12]:
arr = np.array(allVectors,dtype=object)
arrNames = np.array(allVectorsFilenames)

In [14]:
arr.shape

(38180,)

In [13]:
np.save("../data/notebooks-sliced-doc2vec-vectors.npy", arr)
np.save("../data/notebooks-sliced-doc2vec-vectors-filenames.npy", arrNames)